# Lab 1.2 - Basic Data Pipeline with ArcPy and psycopg2
### Luke Zaruba
### GIS 5572: ArcGIS II
### 2023-02-16

In the following notebook, a basic data pipeline that, (1) creates a polygon using ArcPy primitives, (2) converts the polygon to WKT, and (3) inserts the aforementioned polygon into a PosgreSQL database using psycopg2.

The pipeline then continues in additional Python files that run a basic Flask app, which allows users to access the data.

In [2]:
import arcpy
import psycopg2

### 1. Creating a Polygon using ArcPy Primitives

In [3]:
# Create List of Points - using Lat/Long (WGS84)
myInputPoints= [arcpy.Point(-94, 44.5), arcpy.Point(-94, 45.5), arcpy.Point(-92.5, 45.5), arcpy.Point(-92.5, 44.5), arcpy.Point(-94, 44.5)]

# Convert to Polygon
myPolygon = arcpy.Polygon(arcpy.Array(myInputPoints))

# Get ArcPy SR and Assign to Polygon
wgs84 = arcpy.SpatialReference(4326)

myPolygon.SR = wgs84

### 2. Convert to WKT

In [4]:
wkt = myPolygon.WKT

### 3. Insert Polygon into Postgres

In [5]:
def executeCreateTable(table):
    # Set Query
    query = f"CREATE TABLE IF NOT EXISTS {table} (objectid SERIAL PRIMARY KEY, geom GEOMETRY)"

    # Try to Execute
    try:
        # Execute Query
        cursor.execute(query)

        # Commit to DB
        connection.commit()

        # Print if Success
        print("Successful excecution")

    except Exception as e:
        # Roll Back Transaction if Invalid Query 
        connection.rollback()

        # Display Error
        print("Error: " + e)

# Function to Execute an Insert Query
def executeInsert(table, polygon_wkt, epsg):
    # Set Query
    query = f"INSERT INTO {table} (geom) VALUES (ST_GeomFromText('{polygon_wkt}', {epsg}))"
    
    # Try to Execute
    try:
        # Execute Query
        cursor.execute(query)
        
        # Commit to DB
        connection.commit()

        # Print if Success
        print("Successful excecution")

    except Exception as e:
        # Roll Back Transaction if Invalid Query 
        connection.rollback()

        # Display Error
        print("Error: " + e)

In [6]:
# Make a Connection to the DB - CHANGE PARAMS TO EXECUTE
connection = psycopg2.connect(
    host="instance_ip_here",
    database="db_name",
    user="user",
    password="pw"
)

# Create Cursor
cursor = connection.cursor()

# Create Table
table_name = "lab1_polygon"

executeCreateTable(table_name)

# Call Execute Function
executeInsert(table_name, wkt, 4326)

# Close Connection
connection.close()

Successful excecution
Successful excecution
